In [1]:
%run training_functions.ipynb

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-15 12:28:32.043451: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744720112.060132 3565164 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744720112.065213 3565164 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744720112.130977 3565164 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 

{'record': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
['train', 'validation', 'test']
['train', 'validation', 'test']
In, Out, get_ipython, exit, quit, open, accelerate, datasets, evaluate, math, np, os, peft, pickle, pytest, ipytest, pd, transformers, torch, time, threading, load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names, LoftQConfig, LoraConfig, get_peft_model, AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer, SFTTrainer, SFTConfig, @py_builtins, @pytest_ar, model_from_pkl, ds_builder1, ds_builder2, ds_builder3, ds_gs, ds_gst1_train, ds_gst1_test, ds_gst2_train, ds_gst2_test, search_with_strings, map_data, print_trainable_parameters, metric, initialize_heartbeat, heartbeat, end_heartbeat, comp_metrics_output, compute_metrics, make_trainer, get_dataframe, get_training_output, 

## Dataset value example 
{'record': 'NZ_LIXJ01000001.1', 'text': 'CGACTCCGCACAGGCCCTAGGCGTGGTCCACGCATCGAGCGCGTGCGCCGTGGCATACACCCGGGCCACCCGTACGACCCTTCCGGCCGCCCCGTGTGCGCCCTCGTCGCCGAGTCCGCTCCGGCGCGCGCGGGCGCACTGCTGTTCCGCGCGGTGTCATGCGCTCCTCCCGTCCCGACATGCGCCTTTTTGCAGGGGAGAGGGAGGGGTAGGGTCGCGCGTACGCCTAGCACGGGGGAACGGTCTTGGGGGACGCAGTGGACGTACGGGTGGCGGCGATCGCCAGTCTCACACCGCTCGAAGAACTCGACAGCGATCCGTTCCTCGTGGACACCCGCAGCCAGCACGACATGTGCGCACGCTGGGCCGCGGACAAGGGGTACGTCGTCACCCGCCAGCTCCGCTTCTACGGGCTGCGCCCCGATCACCACGCCCTGTGGAGCGATGTCGAGAGCGGCGACGTCGAGGTGTTCGTCGCCGCCAACGACCGGGTGCTGGAACGGGCCCTCACGTCGGTCGCGGAGTTCGCCGCGGAGTGCGCGCGGCGCGGCGTGCGTCTGGAGATCGCCGGGCTGGACGAGCCGCCGTACAACGCCGGCACGAAGGCGAGCGTGCATCGCAGGCTCTCCATGCCCACCGCCGGTTACGACGGCCGCTGACGCACTCCGCCTCCACTGCCGGCTCGCGGCCCCTGCCCTCGTCCCGGCCCCGTCCCTTCCGCGGCTTGCCGTCGGGCTGTGACAGGCTGGGGGACAGGGTCCGCAGCGGCGGGGCCCGGACGTGAGGTGGTTGAGGCGTGGGTGACGGGCGATGGCGAACGGTCGGCAACGCCCTGATGCGAGTGGCCGTGGTGTGGGCGGTCTCCACGCTCACGATGCTGGTGCTCGCCGGGGTGCTGCCGGAGTTCCGGCTCCAGTCGGACGACGGCGACAGCATCACCAAGACCGCGTTCACCGCGGCCTGGGGCGCCGGAGCGTTCGGCCTGCTCTCCGCACTCGTC'}

In [2]:
# let's try prompting the fine-tuned models
def continued_convo(model, tokenizer, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", device_map="auto", max_new_tokens=512, model=model, tokenizer=tokenizer)
    if convo==None:
        messages=[{"role": "system", "content": "You are a helpful assistant"}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    answer=pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
    print(answer)
    base_pairs=""
    content=""
    base_pairs=get_pairs(answer) # run the get_pairs method to return just the genome data from a response
    messages.append(answer)
    return pipe, messages, base_pairs

In [3]:
def get_pairs(output):
    base_pairs=""
    content=""
    if ":" in output['content']: #the base pairs are output after a colon
        content=output['content'].split(":")[1] # take all of the string after the colon
        for i in range(len(content)):
            if content[i]=='A' or content[i]=='C' or content[i]=='T' or content[i]=='G': # get the first base pair character
                base_pairs=content[i:] # take the rest of the following pairs
                break # stop the loop when the base pairs are found
        return base_pairs

In [37]:
# this allows prompting where the model is fed some of the training or test data 
# so we see how much more can be extracted
def prompt_with_data(model, tokenizer, genome, pairs):
    pipe=pipeline("text-generation", model=model, tokenizer=tokenizer)
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "What base pairs are present in genome " + genome + "?"},
        {"role": "assistant", "content": "The starting base pairs in " + genome + " are:" + pairs}
    ]
    
    output=pipe(messages, max_new_tokens=512)[0]
    print(output)
    base_pairs=output['generated_text'][-1]['content'].split(":")[1] # extract just the generated pairs
    base_pairs=base_pairs.split(pairs, 1)[1] # split after the base pairs that were fed to the model
    return base_pairs

In [5]:
ds_gst1_test[0]

{'record': 'JAHFAU010000168.1',
 'text': 'CTCCTCGGCCAGCAAGCGCATCGTGAGAACCGGACTGACGATCAGCAAAACGGTCGCCAGCGGGCCCAATACCATGCGGCCATCCGGGTTCATCTGGCCGGTCGGCAGGCCAAAGTTAACGTTTAGGGAGAAGATCAGGCCGAGCACCAGGAAGCTGAGGGCGGCGGCCGCGTAAGCCAGCGGCGACACGAAGTAGCGGCGGAACTCGCGCACTGCAATAACCCAGCTGGTGCGCACGATCTAGCTCTCCTGCGGCACGACTTGCTCTTCCTCGCGAGTCAGCAGCAGGAAGATGCTTTCCAGGCTGACCCCGGCCGTTGTCAGCTCCAGCAAGTCCAGATCGGCTTCAACGATGCGGCGCGCCAACAGCGGGCGCAGGTCGCGGCCGGGCGAAGAGATCACCTCCACGGTGCCTGCCTGCCCGGCAAGCTCGACCGACTCCACTCCCGGGACGGACTTGAGCAGGTCGACCAGATCGGCTGCCGGGGCAGAGCTGCGGATCAGAATCCGGTACGCACCGGCCAGCTGGGCCGAAAGATTGGCGGGCGTATCTTCCGCCACCAACCGCCCCTTGTTGATAATCAGGACTCGATCACAAACTTGCTGTGCTTCCGAGAGGATGTGGGTGGACAACATGACGGTCCGCTCCCGCCCGAGCTCTCGGATTAAGTCCCGGACTTCGATGATTTGGGCCGGATCCAGTCCAATGGTCGGCTCGTCCAGGATCAAGACCTCGGGCCGGTGCAAGAGTGCCTGGGCCAGGCCGACCCGCTGTCGCAGACCTTTCGACAGGTTGCCAATCAGGCTATCACCGCGTTTTTCGAGGCCGACCTGCGTTAGGGATTCCTCGACCCGGCCATCCAGGTCCTCCACCCGCCGAAGCGAACCCATGAACGAGAGGTATTCGCGGCCGGACATCTCCGGGTACAGCGGGACGGTCTCCGGCAGGTATCCGACC

In [6]:
ds_gst1_test[62] # try prompting it with the newer data

{'record': 'IMGPR_plasmid_3300006450_000026',
 'text': 'CAAAGATCACCAGGAACAGCAGCGCCGGGGCGACCCAAAGCCAGCCCTGCAGGCTTTGCCGCCACGTCGCGCGGCTATGATCGGACCTGTCGGTCATCGGTCAGTCCTGTGGGAGAGGGGGAAATGGGCCGGGCGACCTGCTGGACGCCCGGCGTCGGTCCAGCGTGGGAGCTTACCGCTGGGCCAGCGTTTCCTTGGCCGCGGCGCGCACCTTTTCCATCACGGCTTCCGGCGTCTCATCACCCGACAGAAGGCCCTGTCCCGCGGCGAGGATCACGTTGGCCGAGGCGGTGTCCATGTCCGTGTCCAGCCAGAGCGCCATCCCGGCGGCATCTTCAAGCGCCTTGAGGCCCGCAAGAGTTTCCGGCGTGGTGGTCGCTTCCGTGACCGACCCAATCACCGCGGACGTCATCCCAGTCGTCGACACATACTTTGCCGCTTGTTCCGGCGAGGTGATGAAGTTCAAGAATGCCAGCGCCTCGTCAGGATGCTCTGTCTTGGACGAGATCAGGAATCCGTCCGGCGCCCCGGTCAAAAGGTCGTTGCGGCCCGCGGCACCTTCCATCGGCGGCATCGGGAAGAAGCCCCACCCCTGTTCGGCCACCGGGGTGTCGGCCAGCATGTAGAACTCGACCAGCTCGAGATAGACCATCGCCGACCGGCCGGCGATGAACGCACCCCGCGCTTGCTGATGCGAGATCGCGTTGGCGCCGTTGTTGAAATAGCCCTTGTCGCCGAAGGATTTGTACAGCGCCAAAGCCTCGACATAGCCGGGATGTGTGTAAAGCTGGTCAGCCGGAGCGAGCAGTTGATAGTCGGCCTGACGCACGTCGTCGGGAACCATCTTGGCGAAAAGGTCGCCGATGTAATGCGTTGCCGCCCAAGGCGCCTGGTTGCCGTAGGCGATCGGCACTTCACCCGCCGCCAAGAGCGCGTCGTTAAGC

In [7]:
ds_gst1_train[2012]

{'record': 'NZ_LSRP01000058.1',
 'text': 'GTAGACTGGCCAAGGACCTCGAAAACCTGTCAAGAAATGCGCTTGCCTTCCTCAAATTAGCCTCCATCCGCCTCATGCTGCGAAGGCTATGTTCAAAATGAGAAAGTGTTCGGACGGACTCTAAGAGCGATGAAACTACCGGCGAGTCCAGACATTGCCGACTGCCGGATCTTGTGTGCGCACATAATTTTATCCCCCTTATCTTCACTGGACATTACCGTCTACGCACATAGCTTAAGAGATGGCGTTCGTAATCAGCATAAAATCTCATCTCTGGTATGGCCGTCCCATCTTTGCGCGAAGATCGCTGAACTGAGGAAAACATTGGGTGAGCCCGACAAAACGCTTCTTCTGCGCCAGCACTTTGCTTCATCCGAATTGTCGCCGCAGGATGCATTTGCCGGATGGCAGCAAGCCGTGGCTCCCGTCTTCGATGTGGCGCCCGTCGATCCTCACGCACAGTTCGCAGCCGCTCACGATACCTATCACCTTGGCGATATTCTTGTCGGTGGCGGCAGCTTTGATGCGACGGCATTTACCCGGAGCCGGCAGAAGGCGCTGTCGGATGGCATCGATCACGTCATGGTTCAGGTCTATACCGTCGGCGGATACACCGGGATGCTGGATGATCGCCCAATCACCGTGCGTCGCGGCGATCTTGTCACGCTCGACATGGCGCGCGAAACCAGAACGATTTCCGTCGCCTCCACCAATATCACCGTCATCATACCGCGGGAGTTATTGAAAGTTCGCGAGGTTCCGCACGGGCATATGTCGCAGGGGCCGAGGGGCTCTGTAT'}

In [8]:
ds_gst1_train[5733]

{'record': 'CAKUCL010000105.1',
 'text': '...'}

In [9]:
ds_gst1_test[410]

{'record': 'IMGPR_plasmid_2698536519_000009',
 'text': 'CAGCAAATCATTTTGCCGTTTTTGTGCCGCAGCCGTTCCGCGCGATCGACAAACTGGCAGTCGGGGAATCTCACGAAGTGCGCACTTATACAAACTTCACTTCGTTTCGTTTGTCGTGCGGTCTGCGACGCTCAAAAAACCGAGAAACGGCGGGAATTTTCGGGACAGGATGTGTAATTGTAATACCGTCACACGCGACGCTATCACAATTACCACCTGGTCAGGGCTGCGCCCCGACACCCCGCCAACGCCTCACGGCCCGCGAAAAAATTCCGCGTACCGCTCACCGTTCCCGAACTCGTCAAACAGAAAATCGCGGGGCCCCGCGCCTCTGGCGTGGGTTCCGATTTTGTGATTGACGACGGCACGGGGAAAACACTGGGGATGGTGGTTGGTGTCACGGAGTGAGGCCAACCACCTGCAAGACGGTTTTTTGTTGTTCACAAAATCGGGGTGAAAGTCTGTGGATGAGATGGGTGTAAGTGGGTGGGGTTGGGAGGTTTTTGGTTACTGAAAATCCAGCTCAAAATTCAAGCAAATCTCGCGCGTGATCCTTGTATTTATACTTAAGGGATAACTCTAGGATTCAAATTCGCCTCCAGCCCTTGTGCCGCAAGGCCTGAGGGGGGATTTGTCGGAAAATGGTTTTGGAGTTATTCTGTCGGGAGTGGTTGCGTTAAAAGTGACCCCCGCAACGACGGGCATCGTTCGGGGGTGTGACCACCTGCAGAAACTGGATTCATGATGAGAGCAACAGGTGATATGGCAATGATAACACGTCAATCGAGCCATCGCCCAGTATGGGGCGGTGTGCATGCAAAACCCTAAACTCCAGTACCTCACTGATTATTCCCCCTCTGATGCGCCCTGGGACGCACACAGAAGCGTTTCTGACGACGTGGGAGGGATTTACCTGCTTGCGGCCGAATATGAGCACTACGGGG

In [10]:
lla_model=AutoModelForCausalLM.from_pretrained('./fine_tuned/lla/') #lla2 is epoch, lla is steps

In [11]:
lla_token=AutoTokenizer.from_pretrained('fine_tuned/lla_token/')

In [12]:
model, answer, pairs=continued_convo(lla_model, lla_token, 
                              "What are genome record numbers used for?")
model, answer2, pairs=continued_convo(lla_model, lla_token, 
                               "What would a genome record number look like?", answer)
model, answer3, pairs=continued_convo(lla_model, lla_token, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4, pairs=continued_convo(lla_model, lla_token, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000010.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': 'Genome record numbers, also known as GenBank accession numbers, are unique identifiers assigned to each DNA sequence in the GenBank database. These numbers serve several purposes:\n\n1. **Unique identification**: Each genome sequence is assigned a unique identifier, allowing researchers to easily identify and reference specific sequences.\n2. **Access to publicly available data**: GenBank accession numbers provide a way for researchers to access and utilize publicly available genome sequences, which can be used for a variety of purposes, such as genetic analysis, phylogenetic studies, and functional genomics.\n3. **Organization and cataloging**: The GenBank database is organized and cataloged using accession numbers, making it easier to navigate and find specific sequences.\n4. **Tracking of sequence changes**: Accession numbers can be used to track changes in genome sequences over time, such as mutations, deletions, or insertions.\n5. **Interoperabili

Device set to use cuda:0


{'role': 'assistant', 'content': "A genome record number is a unique identifier assigned to a specific DNA sequence. Here's an example of what a genome record number might look like:\n\n**GenBank Accession Number:** AT5G000000010\n\n**GenBank Database:** GenBank (National Center for Biotechnology Information)\n\n**Database ID:** AT5G000000010\n\n**Database Name:** Homo sapiens (Human)\n\n**Record ID:** AT5G000000010_1\n\n**Record Name:** Homo sapiens (Human) Genome (AT5G000000010_1)\n\n**Record Type:** Genome sequence\n\n**Record Description:** Genome sequence of Homo sapiens (Human) at position 1 of the reference genome AT5G000000010.\n\n**Sequence ID:** AT5G000000010_1\n\n**Sequence Length:** 105,111 base pairs\n\n**Accession Number:** AT5G000000010\n\n**Sequence Description:** AT5G000000010_1 is a DNA sequence of 105,111 base pairs, representing the Homo sapiens (Human) genome at position 1.\n\n**Sequence ID:** AT5G000000010_1\n\nThis is a simplified example, but it illustrates the 

Device set to use cuda:0


{'role': 'assistant', 'content': 'To identify the genome, we need to look up the record number "NZ_LIXJ01000001.1" in the GenBank database.\n\nAfter searching the database, I found that the record "NZ_LIXJ01000001.1" corresponds to the genome sequence of the New Zealand feral chicken (Gallus gallus domesticus) at position 1 of the reference genome NZ_LIXJ01000001.\n\nHere\'s the breakdown:\n\n* **GenBank Accession Number:** NZ_LIXJ01000001.1\n* **Database ID:** NZ_LIXJ01000001\n* **Database Name:** New Zealand feral chicken (Gallus gallus domesticus)\n* **Record ID:** NZ_LIXJ01000001.1\n* **Record Name:** New Zealand feral chicken (Gallus gallus domesticus) Genome (NZ_LIXJ01000001.1)\n* **Record Type:** Genome sequence\n* **Record Description:** Genome sequence of New Zealand feral chicken (Gallus gallus domesticus) at position 1 of the reference genome NZ_LIXJ01000001.1.\n\nSo, the genome identified by the record number "NZ_LIXJ01000001.1" is the New Zealand feral chicken (Gallus gall

In [14]:
search_with_strings(ds_gst1_train, 'GAGCTCTACGGCGGTGGCTTGGTGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGG') # this was just an example not in response to a specific genome

('GAGCTCTACGGCGGTG', 16, 1)

In [15]:
model, answer, pairs=continued_convo(lla_model, lla_token, "Please provide a list with ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are ten example base pairs for a genome:\n\n1. A-T (Adenine-Thymine)\n2. C-G (Cytosine-Guanine)\n3. G-C (Guanine-Cytosine)\n4. A-T (Adenine-Thymine)\n5. C-G (Cytosine-Guanine)\n6. G-A (Guanine-Adeine)\n7. A-T (Adenine-Thymine)\n8. C-G (Cytosine-Guanine)\n9. G-C (Guanine-Cytosine)\n10. A-T (Adenine-Thymine)\n\nThese base pairs are commonly found in DNA and are the most stable and conserved across different species.'}


In [18]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGACT"}
]

output=pipe(messages, max_new_tokens=512)[0]['generated_text'][-1]
print(output)
pairs=get_pairs(output)

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are twenty example base pairs for a genome: CGACTGGGGATCGGTGAGCGGCGGATCCGGGGGGAGGCGGGCGGCGAGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGGCTGGCGGG

In [20]:
search_with_strings(ds_gst1_train, pairs)

('CGACTGGGGATCGGTGA', 17, 1)

In [38]:
output=prompt_with_data(lla_model, lla_token, "NZ_LIXJ01000001.1", "CGACTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LIXJ01000001.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in NZ_LIXJ01000001.1 are:CGACTCAGATGGCCGGGGATGACGATCGGCGGGACGCGCCGCGGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATGCGCGGCGGGCGGATGGGAGGATG

('AGATGGCCGGGGAT', 14, 1)

In [39]:
output=prompt_with_data(lla_model, lla_token, "JAHFAU010000168.11", "CTCCTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000168.11?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHFAU010000168.11 are:CTCCTCCTCTTTCGCGGCGCTCTCGCGTCCGCTGCTGCTGCTCGGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTG

('CTCTTTCGCGGCGCT', 15, 1)

In [43]:
# see how much test data the model can extract
output=prompt_with_data(lla_model, lla_token, "JAHFAU010000093.1", "CGATC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000093.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHFAU010000093.1 are:CGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGGATCAGG

('AGGATCAGGATCAGG', 15, 1)

In [44]:
search_with_strings(ds_gst1_test, output)

('AGGATCAGGATC', 12, 4)

In [30]:
model, answer, pairs=continued_convo(lla_model, lla_token, """Please provide five examples for what
    the first 20 base pairs of a genome may look like""")

Device set to use cuda:0


{'role': 'assistant', 'content': 'The first 20 base pairs of a genome are crucial for understanding its structure and function. Here are five examples of what the first 20 base pairs of a genome might look like:\n\n1. **ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG

In [31]:
search_with_strings(ds_gst1_train, pairs)

('ATCGATCGATCGATC', 15, 1)

In [34]:
# prompt it with training data from the new half (the second 500)
output=prompt_with_data(lla_model, lla_token, "JAHEFU010000133.1", "TCGTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHEFU010000133.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:TCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTC

('GTCGTCGTCGTCGTCG', 16, 5)

In [46]:
# now prompt with new test data
output=prompt_with_data(lla_model, lla_token, "IMGPR_plasmid_3300006450_000026", "CAAAG")
print(search_with_strings(ds_gst1_train, output))
search_with_strings(ds_gst1_test, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome IMGPR_plasmid_3300006450_000026?'}, {'role': 'assistant', 'content': 'The starting base pairs in IMGPR_plasmid_3300006450_000026 are:CAAAGAGAAATGGCCCGGAGGCGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGA

('AGAAATGGCCCG', 12, 2)

In [49]:
# try with training data sample 2012
output=prompt_with_data(lla_model, lla_token, "NZ_LSRP01000058.1", "GTAGA")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LSRP01000058.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in NZ_LSRP01000058.1 are:GTAGAAGTCAATGTGACCTTGGGTGGTCTGCGGCTGTTCCCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGC

('AGTCAATGTGACCTT', 15, 1)

In [52]:
output=prompt_with_data(lla_model, lla_token, "CAKUCL010000105.1", "GCCAG")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome CAKUCL010000105.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in CAKUCL010000105.1 are:GCCAGCGAGCTAGGCTGCTGGCTGCGCCGGTGGCCGGCCGGCTGGCCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGC

('CGAGCTAGGCTGC', 13, 1)

In [51]:
# prompt with a later piece of test data (400)
output=prompt_with_data(lla_model, lla_token, "IMGPR_plasmid_2698536519_000009", "CAGCA")
print(search_with_strings(ds_gst1_train, output))
search_with_strings(ds_gst1_test, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome IMGPR_plasmid_2698536519_000009?'}, {'role': 'assistant', 'content': 'The starting base pairs in IMGPR_plasmid_2698536519_000009 are:CAGCAAGCCGGTTGATGGTGTTTGGGCGCAGGCGGATGATGAGGAGGGTGGCGTGTGAGGCGGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGAGGGTGGCGTGGCGGATGATGAGGA

('AGCCGGTTGATGG', 13, 1)

In [ ]:
# the model above is fine-tuned using steps as the eval_strategy

In [53]:
lla_epoch=AutoModelForCausalLM.from_pretrained('./fine_tuned/lla2/') 
lla_token_epoch=AutoTokenizer.from_pretrained('./fine_tuned/lla2_token')

In [55]:
model, answer, pairs=continued_convo(lla_epoch, lla_token_epoch, 
                              "What are genome record numbers used for?")
model, answer2, pairs=continued_convo(lla_epoch, lla_token_epoch, 
                               "What would a genome record number look like?", answer)
model, answer3, pairs=continued_convo(lla_epoch, lla_token_epoch, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4, pairs=continued_convo(lla_epoch, lla_token_epoch, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000001.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': 'Genome record numbers, also known as GenBank accessions or DNA accession numbers, are unique identifiers assigned to each DNA sequence or genome sequence in the GenBank database. These numbers are used for various purposes, including:\n\n1. **Identification and categorization**: Genome record numbers help researchers and scientists identify and categorize different DNA sequences, such as those from viruses, bacteria, or eukaryotes.\n2. **Database searching and retrieval**: When searching for specific DNA sequences or genome sequences, researchers can use the genome record number to quickly locate and retrieve the desired information.\n3. **Data sharing and collaboration**: Genome record numbers facilitate data sharing and collaboration among researchers, as they can easily identify and access specific sequences or genomes.\n4. **Tracking and updating**: As new DNA sequences or genome sequences become available, the corresponding genome record numbers a

Device set to use cuda:0


{'role': 'assistant', 'content': 'A genome record number is typically a unique identifier assigned to a DNA sequence or genome sequence in the GenBank database. The format and structure of a genome record number can vary depending on the specific database or registry being used, but here is a general outline of what a genome record number might look like:\n\n**Example:**\n\n* GenBank accession number: NC_000001.1\n* Sequence ID: ATTC00000000000.1\n* Database ID: NCBI_GenBank\n* Taxonomy ID: NCBI_Taxonomy\n* Release ID: NCBI_Taxonomy_release_123\n* Release date: 2022-01-01\n* Database name: NCBI_GenBank_2022\n\n**Components of a genome record number:**\n\n1. **Database ID**: The unique identifier for the database, such as NCBI_GenBank.\n2. **Taxonomy ID**: The identifier for the taxonomic classification of the organism, such as NCBI_Taxonomy.\n3. **Release ID**: The identifier for the release of the genome sequence, such as NCBI_Taxonomy_release_123.\n4. **Release date**: The date when 

Device set to use cuda:0


{'role': 'assistant', 'content': 'To identify the genome, we need to look up the record number "NZ_LIXJ01000001.1" in the GenBank database.\n\n**GenBank Accession Number Lookup:**\n\nAfter searching the GenBank database, I found that the record number "NZ_LIXJ01000001.1" corresponds to the following DNA sequence:\n\n* **Accession Number:** NZ_LIXJ01000001.1\n* **Sequence ID:** NZ_LIXJ01000001.1\n* **Database ID:** NZ_96000.1\n* **Taxonomy ID:** NZ_96000.1\n* **Release ID:** NZ_96000.1_release_2022\n* **Release date:** 2022-01-01\n* **Database name:** NZ_96000.1_2022\n\nThis DNA sequence is part of the **New Zealand (NZ)** genomic project, which is a collaborative effort between the New Zealand Ministry of Health and the University of Otago.\n\nThe **NZ_LIXJ01000001.1** sequence corresponds to the **New Zealand isolate of the influenza virus**, which is a subtype of the **H7N9** influenza virus.\n\n**Genome Record Number Breakdown:**\n\nBased on the record number "NZ_LIXJ01000001.1", th

In [57]:
search_with_strings(ds_gst1_train, 'ATGCGCTGCGCGGCGCGGCGGCGCGGCGGCGGCGGCGGCGGCG')

('ATGCGCTGCGCGGCG', 15, 5)

In [ ]:
# the model above is fine-tuned using epochs as the eval_strategy

In [ ]:
# check if eval_strategy makes a difference in how much data can be extracted

In [63]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=lla_epoch, tokenizer=lla_token_epoch)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]['generated_text'][-1]
print(output)
pairs=get_pairs(output)

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are twenty example base pairs for a genome: CGACCGGGATCGGCGCCGCGCCGCGGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCGCCGCG'}


In [64]:
search_with_strings(ds_gst1_train, pairs)

('CGACCGGGATCGGCGCCGCG', 20, 1)

In [72]:
output=prompt_with_data(lla_epoch, lla_token_epoch, "JAHFAU010000168.1", "CTCCTC")
print(search_with_strings(ds_gst1_train, output))
search_with_strings(ds_gst1_test, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000168.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHFAU010000168.1 are:CTCCTCAGCCGACAGGGGAGGTGAGGCGATGACAGCGGCGGTGGAGCGCGAGCGCTGGGCGATGGCGGGCGCGGCGAGCGGCGAGCGGGCGAGCGGCGAGCGAGCGGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGGCGAGCGAGCGG

('AGCCGACAGGGGA', 13, 1)

In [74]:
# prompt with some training data
output=prompt_with_data(lla_epoch, lla_token_epoch, "NZ_LIXJ01000001.1", "CGACTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LIXJ01000001.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in NZ_LIXJ01000001.1 are:CGACTCAGTGGTGTAGCGATGGTGGAGCGGAGAAGGCTGCGGAGGAGGCGGAGCGGAGGAGCGGAGGAGGAGCGGAGGAGGAGGAGCGGAGGAGGAGGAGGAGCGGAGGAGGAGGAGGAGGAGGAGGAGCGGAGGAGGAGGAGGAGGAGGAGCGGAGGAGGAGGAGGAGGAGGAGGAGGAGCGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGA

('AGTGGTGTAGCGA', 13, 2)

In [76]:
# prompt with a later piece of test data (400)
output=prompt_with_data(lla_epoch, lla_token_epoch, "IMGPR_plasmid_2698536519_000009", "CAGCA")
print(search_with_strings(ds_gst1_train, output))
search_with_strings(ds_gst1_test, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome IMGPR_plasmid_2698536519_000009?'}, {'role': 'assistant', 'content': 'The starting base pairs in IMGPR_plasmid_2698536519_000009 are:CAGCAAGCAGCGACCGGCGAGCGACGCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCGGACCG

('AGCAGCGACCGG', 12, 3)

In [ ]:
# performance is pretty consistent